# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [4]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## An event data CSV file titled <font color=red>event_datafile_new.csv</font> has been generated, located within the Workspace directory. The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra code is presented in the cells below, Cassandra is manipulated using the cassandra python driver

#### Creating a Cluster to work within

In [2]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace for this project

In [3]:
# Create a Keyspace name 'sparkify'

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace so that queries may be executed in it

In [4]:
# Set KEYSPACE to the keyspace 'sparkify'

try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Creating queries to ask and answer the following three questions of the data

NoSQL databases like Cassandra must create tables and model data based on the queries to be run against them. They are not optimized for ad hoc queries. For the purpose of this project, it is assumed that the analysis group presented the following queries as representative of the queries they intend to run regularly against the data. Thus the tables must be created with them in mind. The example queries are as follows:

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

This query implies the need for a primary key based on session ID and item in sesssion given that they are mentioned explicitly in the example query. Since the item in session values are unique to each session ID, the table should be partitioned by session ID. Item in session should be a clustering column. These two columsn should be sufficient to uniquely identify each row. Additionally the columns asked for should be included in the table.

**session_library**
- sessionId: partition key
- itemInSession: clustering key
- artist
- song
- length

**QUERY**
    
    SELECT artist, song, length
    FROM session_library
    WHERE sessionID = 338 AND itemInSession = 4;

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    
This query implies the need for a primary key based on user ID and session ID given that they are mentioned explicitly in the example query. There is also a need to sort by item in session because songs are uniquely identified in a session by the item in session ID. Thus the partition key should be user ID. This will make each partition unique to each user, creating a log of songs played, ordered by session ID and item in session. The session ID and item in session should be the clustering columns. These columns are sufficient to uniquely identify each row. Additionally the columns asked for should be included in the table.

**user_library**
- userId: partition key
- sessionId: clustering key
- itemIdSession: clustering key
- song
- artist
- firstName
- lastName

**QUERY**

    SELECT artist, song, firstName, lastName
    FROM user_library
    WHERE userID = 10 AND sessionID = 182;
    
### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

This query implies the need for a primary key based on song given that a song name is mentioned explicitly in the example query. Since no other conditional columns are mentioned in the query, song name should act as the partition key. This will create partitions unique to each song, logging the users that play that given song. Adding clustering columns of artist and userId should create a unique primary key for each row in the table.

**song_library**
- song: partition key
- artist: clustering key
- userId: clustering key
- firstName
- lastName

**QUERY**

    SELECT firstName, lastName
    FROM song_library
    WHERE song = 'All Hands Against His Own';
    
## Create queries to generate the tables, load them with data, and test them with the associated queries

For each of the example queries developed above, a table is created. The queries are unique to such a degree that unique tables must be created for each query. The following code creates the tables, loads them with the applicable data, and executes the example select queries provided. 

In [5]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
## 
## CREATE TABLE EXECUTION

query = """CREATE TABLE IF NOT EXISTS session_library (
           sessionID INT
         , itemInSession INT
         , artist TEXT
         , song TEXT
         , length DECIMAL
         , PRIMARY KEY (sessionID, itemInSession)
           )"""
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [6]:
## INSERT DATA INTO TABLES

file = 'event_datafile_new.csv' # event datafile generated from event_data files

with open(file, encoding = 'utf8') as f: # opens event datafile
    csvreader = csv.reader(f) # generates csv object
    next(csvreader) # skips header row
    
    for line in csvreader: # iterates through every row in the event datafile
        
        ## Generates a row insertion statement, injecting applicable values from the event datafile
        query = "INSERT INTO session_library (sessionID, itemInSession, artist, song, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        ## All values in the event datafile are stored as strings, thus some values must be converted to the appropriate
        ## datatypes as dictated by the table definition. Then the query is executed for each row in the event datafile.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [7]:
## EXECUTE TEST SELECT QUERY

query = """SELECT artist, song, length
           FROM session_library
           WHERE sessionID = 338 AND itemInSession = 4;"""

results = session.execute_async(query) # execute_async allows for generation of result() object for printing
rows = results.result()

## print all rows of result table
print("QUERY 1 TEST:")
for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=Decimal('495.3073'))


In [8]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
## 
## CREATE TABLE EXECUTION

query = """CREATE TABLE IF NOT EXISTS user_library (
           userID INT
         , sessionID INT
         , itemInSession INT
         , song TEXT
         , artist TEXT
         , firstName TEXT
         , lastName TEXT
         , PRIMARY KEY (userID, sessionID, itemInSession)
           )"""
try:
    session.execute(query)
except Exception as e:
    print(e)   

In [9]:
## INSERT DATA INTO TABLE

file = 'event_datafile_new.csv' # event datafile generated from event_data files

with open(file, encoding = 'utf8') as f: # opens event datafile
    csvreader = csv.reader(f) # generates csv object
    next(csvreader) # skips header row
    
    for line in csvreader: # iterates through every row in the event datafile
        
        ## Generates a row insertion statement, injecting applicable values from the event datafile
        query = "INSERT INTO user_library (userID, sessionID, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        ## All values in the event datafile are stored as strings, thus some values must be converted to the appropriate
        ## datatypes as dictated by the table definition. Then the query is executed for each row in the event datafile.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))  

In [10]:
## EXECUTE TEST SELECT QUERY

query = """SELECT artist, song, firstName, lastName
           FROM user_library
           WHERE userID = 10 AND sessionID = 182;"""

results = session.execute_async(query) # execute_async allows for generation of result() object for printing
rows = results.result()

## print all rows of result table
print("QUERY 2 TEST:")
for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [15]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
## 
## CREATE TABLE EXECUTION

query = """CREATE TABLE IF NOT EXISTS song_library (
           song TEXT
         , artist TEXT
         , userID INT
         , firstName TEXT
         , lastName TEXT
         , PRIMARY KEY (song, artist, userID)
           )"""
try:
    session.execute(query)
except Exception as e:
    print(e)                       

In [16]:
## INSERT DATA INTO TABLE

file = 'event_datafile_new.csv' # event datafile generated from event_data files

with open(file, encoding = 'utf8') as f: # opens event datafile
    csvreader = csv.reader(f) # generates csv object
    next(csvreader) # skips header row
    
    for line in csvreader: # iterates through every row in the event datafile
        
        ## Generates a row insertion statement, injecting applicable values from the event datafile
        query = "INSERT INTO song_library (song, artist, userID, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        ## All values in the event datafile are stored as strings, thus some values must be converted to the appropriate
        ## datatypes as dictated by the table definition. Then the query is executed for each row in the event datafile
        session.execute(query, (line[9], line[0], int(line[10]), line[1], line[4]))  

In [17]:
## EXECUTE TEST SELECT QUERY

query = """SELECT firstName, lastName
           FROM song_library
           WHERE song = 'All Hands Against His Own';"""

results = session.execute_async(query) # execute_async allows for generation of result() object for printing
rows = results.result()

## print all rows of result table
print("QUERY 3 TEST:")
for row in rows:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [4]:
## Drop the table before closing out the sessions
session.execute("DROP TABLE IF EXISTS session_library;")

In [ ]:
session.execute("DROP TABLE IF EXISTS user_library;")

In [14]:
session.execute("DROP TABLE IF EXISTS song_library;")

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()